<a href="https://colab.research.google.com/github/gyuholee94/NLP/blob/master/08_Prac_2_Keyword_Extraction%EB%B3%B5%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 8장. 핵심 키워드 추출 (Keyword Extraction)

# 8-0 데이터 준비

## Mecab 설치 (필요시)

In [ ]:
!sudo apt-get install g++ openjdk-7-jdk # Install Java 1.7+
!sudo apt-get install python-dev; pip install konlpy     # Python 2.x
!sudo apt-get install python3-dev; pip3 install konlpy   # Python 3.x
!sudo apt-get install curl
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Package openjdk-7-jdk is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source

E: Package 'openjdk-7-jdk' has no installation candidate
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-dev is already the newest version (2.7.15~rc1-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 48 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-dev is already the newest version (3.6.7-1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'sudo apt autoremove' to remove it.
0 upgraded,

In [ ]:
import Mecab
m = Mecab.tagger()
out = m.parse('이규호는 활용 핵심키워드를 추출한다.')
print(out)

ModuleNotFoundError: ignored

In [ ]:
from konlpy.tag import Hannanum, Kkma, Komoran, Mecab, Okt, Twitter
hannanum = Hannanum()
kkma = Kkma()
komoran = Komoran()
mecab = Mecab()
okt = Okt()
twitter = Twitter()

/usr/local/lib/python3.6/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


# 8-1 TF-IDF 활용 핵심키워드 추출

## 실습 1. sklearn 활용


In [ ]:
import requests 
from bs4 import BeautifulSoup


def get_news_by_url(url):
  resp = requests.get(url)
  soup = BeautifulSoup(resp.content, 'lxml')
  a_tag = soup.select('div#articleBodyContents')
  a_tag_text = a_tag[0].text.replace('\n', ' ')
  a_tag_text = a_tag_text.replace('// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}', ' ')
  # a_tag_text = [a.text for a in a_tag]
  
  return a_tag_text

docs = []
docs.append(get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=018&aid=0004430108') )
docs.append(get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=001&aid=0011614790') )
docs.append(get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=014&aid=0004424362') )
docs.append(get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=119&aid=0002402191') )
docs.append(get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=030&aid=0002882728') )
len(docs)

5

### 1) 전처리

In [ ]:
docs

['        과기정통부, 22일 유영민 장관 등 참석해 기념행사2021년까지 1516억원 투입, 5100여종 데이터 구축민간 클라우드 통한 외부연계체계도.."개방성 강화"[이데일리 이재운 기자] 국가 차원의 빅데이터 활용 시대가 열린다. 새로운 산업 창출과 기존 산업의 변화에 이르는 ‘혁신성장’을 위한 센터가 문을 연다. 10개 분야에 걸쳐 ‘데이터 경제’의 발전을 위한 정부의 청사진을 현실로 구현하는데 앞장선다는 계획이다.22일 과학기술정보통신부는 서울 중구 대한상공회의소에서 데이터 생태계 조성과 혁신 성장의 기반 마련을 위한 ‘빅데이터 플랫폼 및 센터’ 출범식 행사를 개최했다. 유영민 과기정통부 장관을 비롯해 노웅래 국회 과학기술정보방송통신위원회 위원장 등 300여명이 참가했다.◇10개 분야 100개 센터..3년간 1516억원 투입이미지: 픽사베이빅데이터는 데이터 활용을 통해 혁신성장을 이루자는 문재인 정부의 경제 성장 핵심 요소중 하나다. 문재인 대통령이 직접 올 들어 데이터 활용과 이에 따른 정보보호(보안)에 대한 중요성을 강조하기도 했다.이런 맥락 속에서 빅데이터센터는 공공과 민간이 협업해 활용도 높은 양질의 데이터를 생산·구축하고, 플랫폼은 이를 수집·분석·유통하는 역할을 담당한다. 과기정통부는 분야별 플랫폼 10개소와 이와 연계된 기관별 센터 100개소를 구축하는데 3년간 총 1516억원을 투입할 계획이며, 올해 우선 640억원 규모의 사업을 추진하고 있다.대상 분야는 △금융(BC카드) △환경(한국수자원공사) △문화(한국문화정보원) △교통(한국교통연구원) △헬스케어(국립암센터) △유통·소비(매일방송) △통신(KT) △중소기업(더존비즈온) △지역경제(경기도청) △산림(한국임업진흥원) 등으로 현재 1차 공모를 통해 72개 빅데이터 센터를 선정했고, 다음달 8일까지 2차 공모를 통해 28개를 추가 선정해 총 100개를 지원, 운영할 계획이다. 이를 통해 데이터 생태계를 혁신하고 기업의 경쟁력을 제고하는 역할을 수행한다.주요 활용 전략·사례를 보면 

In [ ]:
mecab?

In [ ]:
mecab.pos(docs[0])

[('과기', 'NNG'),
 ('정통부', 'NNG'),
 (',', 'SC'),
 ('22', 'SN'),
 ('일', 'NNBC'),
 ('유영민', 'NNP'),
 ('장관', 'NNG'),
 ('등', 'NNB'),
 ('참석', 'NNG'),
 ('해', 'XSV+EC'),
 ('기념행사', 'NNG'),
 ('2021', 'SN'),
 ('년', 'NNBC'),
 ('까지', 'JX'),
 ('1516', 'SN'),
 ('억', 'NR'),
 ('원', 'NNBC'),
 ('투입', 'NNG'),
 (',', 'SC'),
 ('5100', 'SN'),
 ('여종', 'NNG'),
 ('데이터', 'NNG'),
 ('구축', 'NNG'),
 ('민간', 'NNG'),
 ('클라우드', 'NNP'),
 ('통한', 'VV+ETM'),
 ('외부', 'NNG'),
 ('연계', 'NNG'),
 ('체계', 'NNG'),
 ('도', 'JX'),
 ('.', 'SF'),
 ('."', 'SY'),
 ('개방', 'NNG'),
 ('성', 'XSN'),
 ('강화', 'NNG'),
 ('"[', 'SY'),
 ('이', 'JKS'),
 ('데일리', 'NNP'),
 ('이재운', 'NNP'),
 ('기자', 'NNG'),
 (']', 'SSC'),
 ('국가', 'NNG'),
 ('차원', 'NNG'),
 ('의', 'JKG'),
 ('빅', 'NNG'),
 ('데이터', 'NNG'),
 ('활용', 'NNG'),
 ('시대', 'NNG'),
 ('가', 'JKS'),
 ('열린다', 'VV+EF'),
 ('.', 'SF'),
 ('새로운', 'VA+ETM'),
 ('산업', 'NNG'),
 ('창출', 'NNG'),
 ('과', 'JC'),
 ('기존', 'NNG'),
 ('산업', 'NNG'),
 ('의', 'JKG'),
 ('변화', 'NNG'),
 ('에', 'JKB'),
 ('이르', 'VV'),
 ('는', 'ETM'),
 ('‘', 'SY')

In [ ]:
from konlpy.tag import Mecab
mecab = Mecab()
preprocessed_docs = []
pre_docs = []
for doc in docs :
  m = mecab.pos(doc)
  for i in range(len(m)):
    if m[i][1][0] =='N':
      preprocessed_docs.append(m[i][0])
    elif m[i][1] =='VV':
      preprocessed_docs.append(m[i][0])
  pre_docs.append(' '.join(preprocessed_docs))

  # pre_docs.append(' '.join([token for token in mecab.pos(doc) if token[1][0] == 'N' or token[1] == 'VV']))
print(len(pre_docs))

  # 명사와 동사만으로 문서 전처리
  
# preprocessed_docs[0][:100]

5


In [ ]:
from konlpy.tag import Mecab
mecab = Mecab()

preprocessed_docs = []
for doc in docs :
  # 명사와 동사만으로 문서 전처리
  preprocessed_docs.append(' '.join([token[0] for token in mecab.pos(doc) if token[1][0] == "N" or token[1] == 'VV']))

print(len(preprocessed_docs))
preprocessed_docs[4][:100]

5


'금융 결제 원 신용 정보원 등 지정 비씨 카드 추가 포함 여부 논의 시행령 발표 후 추진 하반기 데이터 유통 판매 급물살 정부 금융 결제 원 신용 정보원 등 업 곳 데이터 중계 기'

### 2) TF-IDF 계산

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(max_df=0.85, max_features=10000)
DTM = count_vect.fit_transform(preprocessed_docs)
list(count_vect.vocabulary_.keys())[:10]
# tfidf_tranf = TfidfTransformer()
# tfidf = tfidf_tranf.fit_transform(DTM)
# tfidf.todense()


['과기', '정통부', '유영민', '장관', '참석', '기념행사', '투입', '여종', '구축', '민간']

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer(smooth_idf=True)
tfidf_transformer.fit(DTM)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [ ]:
# import pandas as pd
# vocab = count_vect.get_feature_names()
# pd.DataFrame(tfidf.todense(), columns = vocab)


### 3) 핵심키워드 추출

In [ ]:
def sort_keywords(keywords):
  return sorted(zip(keywords.col, keywords.data), key=lambda x: (x[1], x[0]), reverse=True)
  
 
def extract_keywords(feature_names, sorted_keywords, n=5):
    #키워드 추출
    extracted_keywords = []
    for (idx, score) in sort_keywords[:n]:
      extracted_keywords.append((feature_names[idx], score))
    return extracted_keywords

In [ ]:
def sort_keywords(keywords):
  return sorted(zip(keywords.col, keywords.data), key=lambda x: (x[1], x[0]), reverse=True)
  
 
def extract_keywords(feature_names, sorted_keywords, n=5):
    #키워드 추출

    return [(feature_names[idx], score) for (idx, score) in sorted_keywords[:n]]      



In [ ]:
def extract_keywords(feature_names, sorted_keywords, n=5):
  return [(feature_names[idx], score) for (idx, score) in sort_keywords[:n]]
  

In [ ]:
doc = preprocessed_docs[0] # 핵심키워드 추출할 문서 조회

feature_names = count_vect.get_feature_names() # TF-IDF 단어 목록
# print(feature_names)
tf_idf_vector = tfidf_transformer.transform(count_vect.transform([doc])) # 문서의 tf-idf 추출
# print(tf_idf_vector)
sorted_keywords = sort_keywords(tf_idf_vector.tocoo()) # TF-IDF를 기준으로 역순 정렬

# 사용자가 지정한 갯수만큼 키워드 추출
keywords = extract_keywords(feature_names, sorted_keywords, 5)

print("\n===== 원문 =====")
print(docs[0][:100])
print("\n=== 핵심키워드 ===")
for k in keywords:
    print(k)


===== 원문 =====
        과기정통부, 22일 유영민 장관 등 참석해 기념행사2021년까지 1516억원 투입, 5100여종 데이터 구축민간 클라우드 통한 외부연계체계도.."개방성 강화"[이데일

=== 핵심키워드 ===
('플랫', 0.2623579798457269)
('계획', 0.2248782684391945)
('정통부', 0.1873985570326621)
('과기', 0.1873985570326621)
('혁신', 0.15119197585668623)
